In [ ]:
import os
from os import listdir
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from PIL import Image
from shutil import copyfile, move
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import moviepy.editor as moviepy
os.getcwd()

## Custom Transformer
Does the following:

1. Convert coloured images to grayscale
2. Crop images according to facial features using haar cascade classifier
3. Resize image accordingly

In [ ]:
class CustomTransform(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
        
    def __call__(self, img):
        opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2GRAY)
        tripleGray = np.stack((gray,)*3, axis=-1)
        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = faceCascade.detectMultiScale(
                tripleGray,
                scaleFactor=1.3,
                minNeighbors=3,
                minSize=(30, 30)
        )
        for (x, y, w, h) in faces:
            if len(faces)==1:
                tripleGray[y:y+h, x:x+w]
        if isinstance(self.output_size, int):
            resized = cv2.resize(tripleGray, (self.output_size, self.output_size))
        if isinstance(self.output_size, tuple):
            resized = cv2.resize(tripleGray, self.output_size)
        return resized

## Transforming Datasets
The following code applies the custom transformation to the specified dataset, transforming and storing the data as tensors. The initial datset should be in a folder with the following subfolders:
- anger
- disgust
- fear
- happy
- neutral
- sadness
- surprise

In [ ]:
# CK+
source_dir = os.getcwd()+'\\datasets\\CK+'
target_dir = os.getcwd()+'\\datasets\\CK+ tensors\\'

data_transforms = transforms.Compose([
        CustomTransform(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

count = 0
image_dataset = datasets.ImageFolder(source_dir, data_transforms)   
for tensor in tqdm(image_dataset):
    torch.save(tensor, target_dir+str(count)+'.pt')
    count+=1

In [ ]:
# FER2013
source_dir = os.getcwd()+'\\datasets\\FER2013'
target_dir = os.getcwd()+'\\datasets\\FER2013 tensors\\'

data_transforms = {
    'train': transforms.Compose([
        CustomTransform(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'test': transforms.Compose([
        CustomTransform(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
}

count = 0
image_datasets = {x: datasets.ImageFolder(os.path.join(source_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}

# for tensor in tqdm(image_datasets['train']):
#     torch.save(tensor, target_dir+'train\\'+str(count)+'.pt')
#     count+=1

count = 0
for tensor in tqdm(image_datasets['test']):
    torch.save(tensor, target_dir+'test\\'+str(count)+'.pt')
    count+=1 
    

## AffectNet dataset
The transformation of the dataset for AffectNet is different from the rest of the datasets. The emotional labels for the dataset is stored separately in a `automatically_annotated.csv`, which is referenced to create subfolders with image copies where the normal transformation steps can be applied.

In [ ]:
# AffectNet - sort images into respective label folders
source_dir = os.getcwd()+'\\datasets\\AffectNet\\Automatically_Annotated_compressed\\Automatically_Annotated\\Automatically_Annotated_Images\\'
target_dir = os.getcwd()+'\\datasets\\AffectNet\\Automatically_Annotated_compressed\\Automatically_Annotated\\AffectNet_sorted\\'
label_dir = os.getcwd()+ '\\datasets\\AffectNet\\Automatically_Annotated_compressed\\Automatically_Annotated\\automatically_annotated.csv'

df = pd.read_csv(label_dir)
for index, row in tqdm(df.iterrows()):

    imageName = row['subDirectory_filePath'].split("/")
    if row['expression']==0:
        copyfile(source_dir+row['subDirectory_filePath'], target_dir+"\\neutral\\"+imageName[1])
    if row['expression']==1:
        copyfile(source_dir+row['subDirectory_filePath'], target_dir+"\\happy\\"+imageName[1])
    if row['expression']==2:
        copyfile(source_dir+row['subDirectory_filePath'], target_dir+"\\sadness\\"+imageName[1])
    if row['expression']==3:
        copyfile(source_dir+row['subDirectory_filePath'], target_dir+"\\suprise\\"+imageName[1])
    if row['expression']==4:
        copyfile(source_dir+row['subDirectory_filePath'], target_dir+"\\fear\\"+imageName[1])
    if row['expression']==5:
        copyfile(source_dir+row['subDirectory_filePath'], target_dir+"\\disgust\\"+imageName[1])
    if row['expression']==6:
        copyfile(source_dir+row['subDirectory_filePath'], target_dir+"\\anger\\"+imageName[1])

In [ ]:
source_dir = os.getcwd()+'\\datasets\\AffectNet\\Automatically_Annotated_compressed\\Automatically_Annotated\\AffectNet_sorted\\'
target_dir = os.getcwd()+'\\datasets\\AffectNet\\Automatically_Annotated_compressed\\Automatically_Annotated\\tensors\\'

data_transforms = transforms.Compose([
        CustomTransform(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

count = 0
image_dataset = datasets.ImageFolder(source_dir, data_transforms)   
for tensor in tqdm(image_dataset):
    if count>302887:
        torch.save(tensor, target_dir+str(count)+'.pt')
    count+=1

### AFF-Wild dataset

The AFF-Wild dataset is different from the rest as it is a dataset of videos instead of images. As such we extracted out individual frames with an interval of 1 second <sup>1</sup>. The emotional label for the specific frame is referenced from the the `annotations` folder. The extracted images are saved in subfolders, where the normal transformation steps can be applied

1. 30 frames constitute 1 second of the video

In [ ]:
# AFF-Wild
def videoToImage(fileName):
    label_data_dir = os.getcwd()+"\\datasets\\AFF Wild\\annotations\\Validation_Set\\"
    vid_data_dir = os.getcwd()+"\\datasets\\AFF Wild\\videos\\Validation_Set\\"
    img_data_dir = os.getcwd()+"\\datasets\\AFF Wild\\images\\Test_Set\\"

    file = open(label_data_dir+fileName, "r")
    imgLabels = []
    frameIteration = 30
    count = 0
    for x in file:
        if(x[0]!='N' and x[0]!='-'):
            if frameIteration == 30:
                imgLabels.append({'frame': count, 'label': x[0]})
                frameIteration = 0
                count+=1
            frameIteration+=1

    # Get specific frames and store as images
    images = []
    cap = cv2.VideoCapture(vid_data_dir+fileName[:-4]+".mp4")
    for x in imgLabels:
        cap.set(1, x['frame'])
        ret, frame = cap.read()
        if(ret):
            images.append(frame)

    # Store images into a separate folder
    labels = ['neutral', 'anger', 'disgust', 'fear', 'happy', 'sadness', 'suprise']
    count = 0
    for x in images:
        img = Image.fromarray(x, 'RGB')
        img.save(img_data_dir+labels[int(imgLabels[count]['label'])]+'\\'+fileName[:-4]+'-'+str(count)+'.png')
        count+=1

In [ ]:
# Loop through all video files
data_dir = os.getcwd() + "\\datasets\\AFF Wild\\annotations\\Validation_Set\\"
files = os.listdir(data_dir)
for file in tqdm(files):
    videoToImage(file)

In [ ]:
source_dir = os.getcwd()+'\\datasets\\AFF Wild\\images\\'
target_dir = os.getcwd()+'\\datasets\\AFF Wild\\tensors\\'

data_transforms = {
    'Train_Set': transforms.Compose([
        CustomTransform(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'Test_Set': transforms.Compose([
        CustomTransform(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
}

count = 0
image_datasets = {x: datasets.ImageFolder(os.path.join(source_dir, x),
                                          data_transforms[x])
                  for x in ['Train_Set', 'Test_Set']}

for tensor in tqdm(image_datasets['Train_Set']):
    torch.save(tensor, target_dir+'train\\'+str(count)+'.pt')
    count+=1

count = 0
for tensor in tqdm(image_datasets['Test_Set']):
    torch.save(tensor, target_dir+'test\\'+str(count)+'.pt')
    count+=1 

### GEMEP dataset

The GEMEP dataset is different from the rest as it is a dataset of videos instead of images. As such we extracted out individual frames with an interval of 1 second <sup>1</sup>. The emotional label for the specific frame is referenced from the the video filename. The extracted images are saved in subfolders, where the normal transformation steps can be applied

1. 25 frames constitute 1 second of the video

In [ ]:
# GEMEP
def videoToImage(fileName):
    source_dir = os.getcwd()+'\\datasets\\GEMEP_Coreset Face Voice\\mp4\\'
    target_dir = os.getcwd()+'\\datasets\\GEMEP_Coreset Face Voice\\images\\'

    counter = 0;

    # Get specific frames and store as images
    images = []
    frameIteration = 0
    cap = cv2.VideoCapture(source_dir+fileName[:-4]+".mp4")
    for x in range(0, int(cap.get(7))):
        if frameIteration%25==0:
            cap.set(1, frameIteration)
            ret, frame = cap.read()
            if(ret):
                images.append(frame)

    # Store images into a separate folder
    labels = ['neutral', 'anger', 'disgust', 'fear', 'happy', 'sadness', 'suprise']
    label = 'none'
    if fileName[2:5] == 'ang':
        label = 'anger'
    elif fileName[2:5] == 'dis':
        label = 'disgust'
    elif fileName[2:5] == 'fea':
        label = 'fear'
    elif fileName[2:5] == 'sad':
        label = 'sadness'
    elif fileName[2:5] == 'sur':
        label = 'suprise'
    count = 0
    for x in images:
        if label == 'none':
            break
        img = Image.fromarray(x, 'RGB')
        img.save(target_dir+label+'\\'+fileName+'-'+str(count)+'.png')
        count+=1

In [ ]:
# Loop through all video files
data_dir = os.getcwd() + "\\datasets\\GEMEP_Coreset Face Voice\\avi\\"
files = os.listdir(data_dir)
for file in files:
    source_dir = os.getcwd()+'\\datasets\\GEMEP_Coreset Face Voice\\avi\\'
    target_dir = os.getcwd()+'\\datasets\\GEMEP_Coreset Face Voice\\mp4\\'
    clip  = moviepy.VideoFileClip(source_dir+file)
    clip.write_videofile(target_dir+file[:-3]+"mp4")

In [ ]:
# Loop through all video files
data_dir = os.getcwd() + "\\datasets\\GEMEP_Coreset Face Voice\\mp4\\"
files = os.listdir(data_dir)
for file in tqdm(files):
    videoToImage(file)

In [ ]:
source_dir = os.getcwd() + "\\datasets\\GEMEP_Coreset Face Voice\\images\\"
target_dir = os.getcwd() + "\\datasets\\GEMEP_Coreset Face Voice\\tensors\\"

data_transforms = transforms.Compose([
        CustomTransform(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

count = 0
image_dataset = datasets.ImageFolder(source_dir, data_transforms)   
for tensor in tqdm(image_dataset):
    torch.save(tensor, target_dir+str(count)+'.pt')
    count+=1

In [ ]:
# IASLab
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
source_dir = os.getcwd() + "\\datasets\\IASLab\\images\\"
target_dir = os.getcwd() + "\\datasets\\IASLab\\tensors\\"

data_transforms = transforms.Compose([
        CustomTransform(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

count = 0
image_dataset = datasets.ImageFolder(source_dir, data_transforms)   
for tensor in tqdm(image_dataset):
    if count>451:
        torch.save(tensor, target_dir+str(count)+'.pt')
    count+=1